In [2]:
#Import libraries
%matplotlib inline
import os
import pandas as pd

import matplotlib.image as mpimg
import matplotlib.pyplot as plt
import numpy as np
from PIL import Image
import tensorflow as tf
import glob
import numpy as np
import sklearn as sk
from scipy.misc import imresize

In [3]:
#Load images
phototexture    = []

folder=os.getcwd()+'/photos/'+'texture/'
texture_dict = {}

filelist = glob.glob(folder+'/*.png')
textureimages = np.array([imresize(np.array(Image.open(fname).convert('RGBA')),10) for fname in filelist])

In [4]:
texturefiles = np.array([np.array(fname.split('/')[-1].replace('.png','')) for fname in filelist])
texturefiles = list(map(int, texturefiles))

In [5]:
texture = pd.read_csv('texture.csv')
texture = texture.drop(['Unnamed: 0'],axis=1)
texture['texture'] = texture.texture.str.replace(',','.')
texture['texture'] = pd.to_numeric(texture['texture'])
texture['inx']   = texture.index

In [6]:
texturefiles = pd.DataFrame(texturefiles,columns=['inx'])

In [9]:
texturedata = texturefiles.merge(texture,on='inx')

In [10]:
texturedata = texturedata.drop(['filename','empresa','gravel_size','image','inx'],axis=1)

In [13]:
#texturedata['category'] = pd.cut(texturedata.texture,bins=[-1000,0,165,275,350,460,20000],labels=['Error', 'Polido',
#                                             'Macio', 'Baixa Rugosidade', 'Rugosidade Moderada','Alta Rugosidade'])
texturedata['category'] = pd.cut(texturedata.texture,bins=[-1000,0,165,275,350,460,20000],labels=[1, 2, 3, 4, 5,6])

In [15]:
X_train = textureimages[0:100]
y_train = texturedata['category'][0:100]